In [1]:
import os
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [1]:
!pip install colab-xterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 2.1 MB/s eta 0:00:00


In [2]:
%load_ext colabxterm

In [ ]:
%xterm

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download#https://github.com/jmorganca/ollama/releases/download/v0.1.27#' | sh

In [3]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [2]:
!ollama pull llava &>/dev/null

In [5]:
!ollama pull mistral &>/dev/null

In [14]:
!ollama list

[GIN] 2024/05/19 - 20:18:29 | 200 |      31.266µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/05/19 - 20:18:29 | 200 |     544.247µs |       127.0.0.1 | GET      "/api/tags"
NAME         	ID          	SIZE  	MODIFIED       
llama2:latest	78e26419b446	3.8 GB	12 minutes ago	


In [5]:
!ollama --version

/bin/bash: line 1: ollama: command not found


In [ ]:
!pip install ngrok
!pip install pyngrok

In [5]:
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="2ggno27BakF83etx7SZkZM7eHWE_4Pvjwv2LPvkytbiQEeXRV"
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [6]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

In [7]:
# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

Ngrok tunnel established at: NgrokTunnel: "https://06f8-34-138-186-132.ngrok-free.app" -> "http://localhost:11434"


In [8]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

In [10]:
! export OLLAMA_HOST=https://06f8-34-138-186-132.ngrok-free.app/

In [ ]:
!ollama pull llama2

In [ ]:
!curl -X POST https://06f8-34-138-186-132.ngrok-free.app/api/generate -d '{ "model": "llama2", "prompt":"Good Morning" , "stream": false }'